import dependies

In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics
import os

Download dataset from kaggle

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download asaniczka/tmdb-movies-dataset-2023-930k-movies

Dataset URL: https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies
License(s): ODC Attribution License (ODC-By)
 51% 113M/220M [00:00<00:00, 1.18GB/s]
100% 220M/220M [00:00<00:00, 1.16GB/s]


In [4]:
!unzip tmdb-movies-dataset-2023-930k-movies.zip -d movie_data

Archive:  tmdb-movies-dataset-2023-930k-movies.zip
  inflating: movie_data/TMDB_movie_dataset_v11.csv  


Data precossing

In [5]:
df = pd.read_csv('movie_data/TMDB_movie_dataset_v11.csv')

In [6]:
df.shape

(1262362, 24)

In [7]:
df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [8]:
df.iloc[1]

,1
id,157336
title,Interstellar
vote_average,8.417
vote_count,32571
status,Released
release_date,2014-11-05
revenue,701729206
runtime,169
adult,False
backdrop_path,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1262362 entries, 0 to 1262361
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   id                    1262362 non-null  int64  
 1   title                 1262349 non-null  object 
 2   vote_average          1262362 non-null  float64
 3   vote_count            1262362 non-null  int64  
 4   status                1262362 non-null  object 
 5   release_date          1019885 non-null  object 
 6   revenue               1262362 non-null  int64  
 7   runtime               1262362 non-null  int64  
 8   adult                 1262362 non-null  bool   
 9   backdrop_path         325703 non-null   object 
 10  budget                1262362 non-null  int64  
 11  homepage              131780 non-null   object 
 12  imdb_id               632346 non-null   object 
 13  original_language     1262362 non-null  object 
 14  original_title        1262349 non-

In [10]:
df.isnull().sum()

,0
id,0
title,13
vote_average,0
vote_count,0
status,0
release_date,242477
revenue,0
runtime,0
adult,0
backdrop_path,936659


In [11]:
df.dropna(subset=['overview', 'keywords', 'tagline', 'original_title', 'genres'], inplace=True)
print(df.isnull().sum())
print(df.shape)

id                          0
title                       0
vote_average                0
vote_count                  0
status                      0
release_date             3397
revenue                     0
runtime                     0
adult                       0
backdrop_path           23364
budget                      0
homepage                57119
imdb_id                 15470
original_language           0
original_title              0
overview                    0
popularity                  0
poster_path              5494
tagline                     0
genres                      0
production_companies    14284
production_countries    11696
spoken_languages         9134
keywords                    0
dtype: int64
(78742, 24)


In [12]:
#fill null with empty string
df.fillna(' ', inplace=True)
df.isnull().sum()

,0
id,0
title,0
vote_average,0
vote_count,0
status,0
release_date,0
revenue,0
runtime,0
adult,0
backdrop_path,0


In [13]:
df = df.drop_duplicates(subset='title', keep='first')
df.shape

(72565, 24)

In [14]:
df = df.reset_index(drop=True)

In [15]:
df['backdrop_path'] = "https://image.tmdb.org/t/p/original" + df['backdrop_path']
df['poster_path'] = "https://image.tmdb.org/t/p/original/" + df['poster_path']
df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,https://image.tmdb.org/t/p/original/8ZTVqvKDQ8...,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,https://image.tmdb.org/t/p/original//oYuLEt3zV...,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,https://image.tmdb.org/t/p/original/pbrkL804c8...,...,Interstellar,The adventures of a group of explorers who mak...,140.241,https://image.tmdb.org/t/p/original//gEU2QniE6...,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,https://image.tmdb.org/t/p/original/nMKdUUepR0...,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,https://image.tmdb.org/t/p/original//qJ2tW6WMU...,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,https://image.tmdb.org/t/p/original/vL5LR6WdxW...,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,https://image.tmdb.org/t/p/original//kyeqWdyUX...,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,https://image.tmdb.org/t/p/original/9BBTo63ANS...,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,https://image.tmdb.org/t/p/original//RYMX2wcKC...,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [16]:
#combaning features
df['soup'] = (df['overview'] + " " +
              df['genres'] + " " +
              df['keywords'] + " " +
              df['tagline']).str.strip()

In [17]:
print(df.shape)
print(df[['title', 'soup']].head())

(72565, 25)
             title                                               soup
0        Inception  Cobb, a skilled thief who commits corporate es...
1     Interstellar  The adventures of a group of explorers who mak...
2  The Dark Knight  Batman raises the stakes in his war on crime. ...
3           Avatar  In the 22nd century, a paraplegic Marine is di...
4     The Avengers  When an unexpected enemy emerges and threatens...


In [18]:
tfidf = tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['soup'])
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2482787 stored elements and shape (72565, 97543)>

In [19]:
consine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [20]:
consine_sim

array([[1.00000000e+00, 5.23412480e-02, 1.76389279e-02, ...,
        4.76352091e-02, 7.88565899e-03, 0.00000000e+00],
       [5.23412480e-02, 1.00000000e+00, 9.60780243e-04, ...,
        9.34626666e-03, 0.00000000e+00, 1.41116886e-02],
       [1.76389279e-02, 9.60780243e-04, 1.00000000e+00, ...,
        0.00000000e+00, 2.19038857e-02, 0.00000000e+00],
       ...,
       [4.76352091e-02, 9.34626666e-03, 0.00000000e+00, ...,
        1.00000000e+00, 3.59370582e-03, 0.00000000e+00],
       [7.88565899e-03, 0.00000000e+00, 2.19038857e-02, ...,
        3.59370582e-03, 1.00000000e+00, 1.17964903e-02],
       [0.00000000e+00, 1.41116886e-02, 0.00000000e+00, ...,
        0.00000000e+00, 1.17964903e-02, 1.00000000e+00]])

In [21]:
df['title_lower'] = df['title'].str.lower()
indices = pd.Series(df.index, index=df['title_lower']).drop_duplicates()

In [22]:
indices

,0
title_lower,
inception,0
interstellar,1
the dark knight,2
avatar,3
the avengers,4
...,...
romance/romance,72560
taromenani,72561
wired to win,72562


In [23]:
list_of_all_titles = df['title_lower'].tolist()

In [80]:
def search_result(title):
  title = difflib.get_close_matches(title, list_of_all_titles)
  if len(title) == 0:
    return 'Movie not found'
  search_result = [indices[i] for i in title]
  return search_result

In [186]:
def recommend(idx, df=df, consine_sim=consine_sim, k=5):
  sim_scores = list(enumerate(consine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:k+1]
  movie_indices = [i[0] for i in sim_scores]
  recommended_df = df.iloc[movie_indices][['title','popularity', 'runtime', 'release_date', 'overview']]
  return recommended_df.sort_values(by='popularity', ascending=False)

In [191]:
sr = search_result(input())
for i in sr:
  print(df.iloc[i]['title'])

master
Master
Masters
Mater


In [190]:
recommend(sr[0], k=10)

,title,popularity,runtime,release_date,overview
13373,No Holds Barred,6.773,93,1989-06-02,Rip is the World Wrestling Federation champion...
14062,Shooting Fish,6.749,109,1997-08-22,Two con artists hire an unwitting medical-scho...
17164,Chain of Command,4.941,88,2015-08-30,After finding his brother murdered after retur...
47245,Scandalous,2.811,92,1984-01-29,An investigative reporter following an espiona...
35577,Mosagallu,2.017,138,2021-03-19,Arjun and Anu are fraternal twins who manage t...
36843,Man Of The People,1.912,81,1937-01-27,An Italian immigrant studying the law gets mix...
48979,Gafla,1.889,0,2006-10-06,Subodh is a man with drive and ambition who se...
57168,Crossover Nexus,1.400,11,2018-10-08,Cartoon Network Heroes need to band together t...
61949,Hunting Corruption - The Network,0.600,54,2011-12-01,The Network is an exclusive group of the most ...
45338,The EndGame,0.600,115,2021-06-13,Will this really be the last round against cor...


export recommendation system

In [84]:
df.to_csv("movies.csv", index=False)

In [85]:
import pickle

In [ ]:
#this take lot of time
with open('consine_sim.pkl', 'wb') as f:
  pickle.dump(consine_sim, f)

In [89]:
def get_file_size(path):
    size_bytes = os.path.getsize(path)
    size_kb = size_bytes / 1024
    size_mb = size_kb / 1024
    size_gb = size_mb / 1024
    return size_gb

In [92]:
file_path = "/content/consine_sim.pkl"
print(f"Size of cosine_sim.pkl: {get_file_size(file_path):.4f} GB")

Size of cosine_sim.pkl: 39.2324 GB


Predicting System in streamlit

In [171]:
!pip install -q streamlit
!pip install pyngrok

In [192]:
%%writefile demo.py
import streamlit as st
import pandas as pd
import difflib
import pickle as pkl

print("Started....")

df = pd.read_csv('movies.csv')
print("Loaded df")

with open('consine_sim.pkl', 'rb') as f:
    consine_sim = pkl.load(f)
print("Loaded cosine sim")

indices = pd.Series(df.index, index=df['title_lower']).drop_duplicates()
list_of_all_titles = df['title_lower'].tolist()

def search_result(title):
  title = difflib.get_close_matches(title, list_of_all_titles)
  if len(title) == 0:
    return 'Movie not found'
  search_result = [indices[i] for i in title]
  return search_result

# Function to get movie recommendations based on index
def recommend(idx, df=df, consine_sim=consine_sim, k=5):
  sim_scores = list(enumerate(consine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:k+1]
  movie_indices = [i[0] for i in sim_scores]
  recommended_df = df.iloc[movie_indices][['title', 'vote_average', 'popularity', 'overview', 'poster_path', 'runtime', 'release_date']]
  return recommended_df.sort_values(by='popularity', ascending=False)

st.set_page_config(
    page_title="Movie Recommender",
    layout="wide",
    page_icon="🎬"
)
st.title("Movie Recommendation System")
movie = st.text_input("Enter name of movie", placeholder="Type a movie name...")

sr_idx = search_result(movie)
if sr_idx == 'Movie not found':
    st.error("Movie not found. Please try another title.")
else:
  st.success(f"Found {len(sr_idx)} results for '{movie}':")
  st.subheader("Click on a movie to get recommendations:")

  cols = st.columns(3)
  for i, idx in enumerate(sr_idx):
      with cols[i]:
          name = df.iloc[idx]['title']
          poster_url = df.iloc[idx]['poster_path']
          st.image(poster_url, caption=name, use_container_width=True)

          if st.button(f"Get Recommendations for {name}", key=i):
              st.success(f"You selected: {name}")
              recommendations = recommend(idx, k=10)
              st.subheader("Recommended Movies:")

              for j, (_, row) in enumerate(recommendations.iterrows()):
                  st.image(row['poster_path'], caption=row['title'], use_container_width=True)
                  st.markdown(f"**Rating**: {row['vote_average']} | **Popularity**: {row['popularity']} | **Runtime** : {row['runtime']} | **Release Date** : {row['release_date']}")
                  st.write(row['overview'])

Overwriting demo.py


In [174]:
from pyngrok import ngrok

ngrok.set_auth_token("2jNfToYiNmhXtMBupcbPJzmezHN_xLkwZCraxCfs7i5sawMS")

# Kill any existing ngrok tunnels
try:
    ngrok.kill()
except:
    pass

# Start Streamlit server
!nohup streamlit run demo.py &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://0cd83ec13ed7.ngrok-free.app
